In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sb 
import matplotlib.pyplot as plt 
import sklearn 
from pandas import Series, DataFrame 
from pylab import rcParams 
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression 
from sklearn.cross_validation import train_test_split 
from sklearn import metrics 
from sklearn.metrics import classification_report

C:\Users\Deepak\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def fill_missing_age(dataset):
    for i in range(1,4):
        median_age=dataset[dataset["Pclass"]==i]["Age"].median()
        dataset["Age"]=dataset["Age"].fillna(median_age)
        return dataset

In [3]:
titanic = pd.read_csv('titanic-train.csv')
columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']
df = pd.DataFrame(titanic, columns=columns)

In [4]:
# fill Missing Age
df = fill_missing_age(df)

# "Sex" Coulumn has male/feamle as value
# to convert these to int. male:1,female:0
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
df['Sex'] = lb.fit_transform(df['Sex'])

In [5]:
X = df.iloc[:,[0,1,2,3,4,5]].values 
y = df.iloc[:,6]

# Splitting data set into Training set & Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/4, random_state=0)

In [6]:
# Feature scaling
from sklearn.preprocessing import  StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test  = sc.fit_transform(X_test)

In [7]:
# Fitting  DecisionTree
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [8]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
acc_score = accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.80      0.70      0.74       139
          1       0.58      0.70      0.64        84

avg / total       0.72      0.70      0.70       223



In [9]:
from sklearn.cross_validation import KFold
cv = KFold(n=len(X), n_folds = 10, random_state=0)

X_fold = df.iloc[:,[0,1,2,3,4,5]] 
y_fold = df.iloc[:,6]

fold_accuracy = []

for train_fold, valid_fold in cv:
    train = X_fold.loc[train_fold]
    valid = X_fold.loc[valid_fold]
    
    train_y = y_fold.loc[train_fold]
    valid_y = y_fold.loc[valid_fold]
    
    model = classifier.fit(X = train, y = train_y)
    valid_acc = model.score(X = train, y = train_y)
    fold_accuracy.append(valid_acc)
    
print("Survival per fold: ", fold_accuracy, "\n")
print("Avg. Survival: ", sum(fold_accuracy)/len(fold_accuracy))

Survival per fold:  [0.982521847690387, 0.9825436408977556, 0.9875311720698254, 0.983790523690773, 0.983790523690773, 0.9825436408977556, 0.983790523690773, 0.983790523690773, 0.9812967581047382, 0.9825436408977556] 

Avg. Survival:  0.9834142795321311


In [10]:
# First stratified K Fold
from sklearn.cross_validation import cross_val_score
score = cross_val_score(estimator = classifier, X=X, y=y, scoring="accuracy", cv=15)
print("Survival: ")
print(score)
print("Avg. Survival: ", score.mean())

Survival: 
[0.71666667 0.8        0.76666667 0.71666667 0.75       0.73333333
 0.88333333 0.8        0.8        0.84745763 0.84745763 0.71186441
 0.84482759 0.77586207 0.94827586]
Avg. Survival:  0.7961607896616664
